In [38]:
import torch
import torchvision
import cv2
from subprocess import call

In [19]:
model = torchvision.models.resnet18(pretrained=True)

In [20]:
network = torch.load("/home/pranjal-ubuntu/Downloads/resnet_model.pt")

In [21]:
smoothed_angle = 0
xs = []
ys = []

In [22]:

img = cv2.imread('steering.jpg')

In [23]:
rows,cols,channels = img.shape

In [24]:
print(rows)
print(cols)

240
240


In [25]:
with open("data.txt") as f:
    for line in f:
        xs.append("driving_dataset" + line.split()[0])
        ys.append(float(line.split()[1]) * scipy.pi / 180)

In [26]:
num_images = len(xs)
print(num_images)

45406


In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [36]:

for i in range(num_images):
    full_image = cv2.imread("driving_dataset/" + str(i) + ".jpg", cv2.IMREAD_COLOR)
    image = cv2.resize(full_image, (256, 256))
    cv2.imshow("image",image)
    image = torch.from_numpy(image)
    image = image.to(device)
    image = image.float()
    image = image.reshape(1, 3, 256, 256)
    predicted = network(image)
    smoothed_angle += 0.3 * pow(abs((predicted - smoothed_angle)), 2.0 / 3.0) * (predicted - smoothed_angle) / abs(predicted - smoothed_angle)
    smoothed_angle = smoothed_angle.item()
    print(f"Predicted steering angle : {smoothed_angle}")
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering", dst)
    cv2.waitKey(1)
    i = i+1


Predicted steering angle : -7.135587692260742
Predicted steering angle : -9.00771713256836
Predicted steering angle : -11.08834171295166
Predicted steering angle : -12.668379783630371
Predicted steering angle : -11.77189826965332
Predicted steering angle : -12.56709098815918
Predicted steering angle : -14.449472427368164
Predicted steering angle : -15.61829948425293
Predicted steering angle : -16.936370849609375
Predicted steering angle : -15.040081024169922
Predicted steering angle : -12.625081062316895
Predicted steering angle : -9.879732131958008
Predicted steering angle : -6.992136001586914
Predicted steering angle : -4.31905460357666
Predicted steering angle : -1.8210725784301758
Predicted steering angle : -0.528947114944458
Predicted steering angle : 0.4102974534034729
Predicted steering angle : 0.00065574049949646
Predicted steering angle : -1.1688168048858643
Predicted steering angle : -2.720254898071289
Predicted steering angle : -3.0013225078582764
Predicted steering angle : 

KeyboardInterrupt: 

In [37]:
cv2.destroyWindow("steering")
cv2.destroyWindow("image")